In [1]:
#импорт
import pandas as pd
import requests
import json
import time
import math
import vk
import collections
import os
from io import open

In [6]:
#подключение api
import foursquare

client_id = '2VRQYSGPD3F1PVQK0MPQI4NZVEJOEXJJMYYFCGHXKIF14FSG'
client_secret = 'PDAGZIQBVJYW2Q03QDMGMQW1XXBOFUHSYOWXINAOQY5N2QKT'
access_token = ''

api = foursquare.Foursquare(client_id=client_id, client_secret=client_secret)

In [3]:
#логер

def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [4]:
#задать сетку

bounds = {'lower':{'lat': 59.808311, 'lng': 30.135219}, 'upper':{'lat': 60.087617, 'lng': 30.526297}} 
radius = 250

def get_circle_centers(lower, upper, radius):
    """
    cover the search area with circles for radar search
    http://stackoverflow.com/questions/7477003/calculating-new-longtitude-latitude-from-old-n-meters
    :param lower: lower bound of area (westmost + southmost)
    :param upper: upper bound of area (eastmost + northmost)
    :param radius: specified radius, adapt for high density areas
    :return: list of circle centers that cover the area between lower/upper
    """
    r, coords = 6378, list()
    while lower[1] < upper[1]:
        tmp = lower[0]

        while tmp < upper[0]:
            coords.append((tmp, lower[1]))
            tmp += (0.25 / r) * (radius / math.pi)
        lower[1] += (0.25 / r) * (radius / math.pi) / math.cos(lower[00] * math.pi / radius)

    return coords


circles = get_circle_centers([bounds["lower"]["lat"], bounds["lower"]["lng"]],  # southwest
                                       [bounds["upper"]["lat"], bounds["upper"]["lng"]],  # northeast
                                       radius)


In [12]:
#search vemues in coord
from itertools import chain

def api_req_search_venues(coord):
    query = {
        'll': '%s,%s' % coord,
        'limit': 50,
    }

    res = api.venues.search(params=query)
    venues = res['venues']
    return venues

def cats(cs):
    return list(map(lambda c: c['name'], cs))

def request_to_list(reqs):
    venues = list(chain.from_iterable(api_reqs))
    venues = dict(map(lambda v: (v['id'], v), venues))
    venues = list(venues.values())
    venues = list(map(
        lambda v: {
            'id':v['id'],
            'name':v['name'],
            'lat': v['location']['lat'],
            'lng': v['location']['lng'],
            'checkinsCount':v['stats']['checkinsCount'],
            'usersCount': v['stats']['usersCount'],
            'categories': cats(v['categories']),
        },
        venues
    ))
    
    return venues


def write_to_file(filename, requests):
    with open(filename, 'a+') as fout:
        venues = request_to_list(requests)
        
        for venue in venues:
            json.dump(venue, fout)
            fout.write(os.linesep)
            
            
            

api_reqs = []
counter = 0
for coord in log_progress(circles, 10):
    if counter % 100 == 0:
        write_to_file('foursqare_venues', api_reqs)
        api_reqs = []
        
    api_reqs.append(api_req_search_venues(coord))
    
write_to_file('foursqare_venues', api_reqs)

VBox(children=(HTML(value=''), IntProgress(value=0, max=8280)))

Error connecting with foursquare API: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/search?ll=59.8738145631258,30.13948830089965&limit=50&v=20160912&client_id=2VRQYSGPD3F1PVQK0MPQI4NZVEJOEXJJMYYFCGHXKIF14FSG&client_secret=PDAGZIQBVJYW2Q03QDMGMQW1XXBOFUHSYOWXINAOQY5N2QKT (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001E7A5638668>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
Error connecting with foursquare API: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/search?ll=59.8738145631258,30.13948830089965&limit=50&v=20160912&client_id=2VRQYSGPD3F1PVQK0MPQI4NZVEJOEXJJMYYFCGHXKIF14FSG&client_secret=PDAGZIQBVJYW2Q03QDMGMQW1XXBOFUHSYOWXINAOQY5N2QKT (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001E7A56381D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo

In [23]:
#file to csv
with open('foursqare_venues', 'rb') as f:
    my_list = [json.loads(line) for line in f]
    
venues = pd.DataFrame(my_list)
venues = venues.drop_duplicates("id")

venues.to_csv('foursqare_venues.csv')

In [2]:
#read from csv
venues = pd.read_csv('foursqare_venues.csv')

C:\Users\Artem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#get main venues category
categories = api.venues.categories()

from anytree import Node, RenderTree, PostOrderIter

def addCatagoriesToTree(categories_list, node):
    for categorie in categories_list:
        categorie_node = Node(categorie['name'], node)
        addCatagoriesToTree(categorie['categories'], categorie_node)
        
def cat_node_list(categorie):
    node = Node(categorie['name'])
    addCatagoriesToTree(categorie['categories'], node)
    return node

def getMainCategorie(tree, node_name):
    if node_name != None or node_name != "":
        for node in PostOrderIter(tree):
            if node.name == node_name:
                return node.path[1]

root = Node('root')
addCatagoriesToTree(categories['categories'], root)

venues['main_categorie'] = venues.categories.apply(lambda c: "" if getMainCategorie(root, c) is None else getMainCategorie(root,c).name)



In [11]:
venues.head()

,Unnamed: 0,Unnamed: 0.1,categories,checkinsCount,id,lat,lng,name,usersCount
0,0,0,['Auto Dealership'],277.0,57613680498e715ad5a8965f,59.808649,30.165717,Вагнер премиум юг (Мерседес),125.0
1,1,1,['Bridge'],3739.0,4fbb1ee6e4b0ffe177c3094d,59.802435,30.168832,Виадук на Таллинском шоссе,2021.0
2,2,2,['Factory'],115.0,50cac9d0e4b054c06e8e6c1b,59.807105,30.136798,"ЗАО ""ОКС 01""",15.0
3,3,3,['Factory'],118.0,4eba2e619911cd1426aa69d3,59.807103,30.136760,"ЗАО ""Пластком""",16.0
4,4,4,['Warehouse'],5.0,54477684498ef2d39d67e1f7,59.809408,30.134302,база орион,2.0


In [13]:
getMainCategorie(root, 'Auto Dealership').name

'Shop & Service'